BYNmasivofy2024*

In [32]:
# Importa las librerias a usar
import pandas as pd
import re
import numpy as np


importar y procesar el K2

In [33]:

# Ubicacion de los archivos a usar
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')


In [34]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

In [35]:
# Cambiamos la hora 24 por 0
kmsmio['Desde'] = kmsmio['Desde'].replace('24', '0')
kmsmio = kmsmio[kmsmio['Desde'] != '0']


In [36]:
# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta'])]), axis=1)


In [37]:
# Se verifica el tamaño del df
kmsmio.shape

(18645, 40)

In [38]:
# Se verifica que la columna id no tenga id repetidos
kmsmio['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

Importar reporte finalizado

In [39]:
# ubicacion del archivo
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'
# Se importa el archivo xlxs
repofi = pd.read_excel(patch2, sheet_name='Programacion')
bitacora = pd.read_excel(patch2, sheet_name='Bitacora')
# Se eliminan las filas y las columnas vacias
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')
bitacora = bitacora.dropna(axis=0, how='all').dropna(axis=1, how='all')


In [40]:
# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']
# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                  str(row['desde']),str(row['hasta'])]), axis=1)


In [41]:
# Se verifica que el id no tenga datos repetidos
repofi['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

In [42]:
# Se verifica el tamaño del df
repofi.shape

(18645, 32)

Se realiza el respectivo Merge con los dos df

In [43]:
# Se realiza el merge con outer en la columna id
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

In [44]:
# Se verifica que el numero de filas en los df sea igual (True), para poder concluir que el merge fue exitoso
merge.shape[0] == kmsmio.shape[0] == repofi.shape[0]

True

In [45]:
merge.shape

(18645, 71)

In [46]:
# Exporta a un archivo excel el Df
#mergebitacora.to_excel('../output/prueba_mergebita.xlsx', index=False, sheet_name='k2_procesado')

Merge con bitacora


In [47]:
merge[['Número de tarea vehículo_x', 'Número de viaje_x']]

,Número de tarea vehículo_x,Número de viaje_x
0,13,1
1,13,2
2,13,3
3,13,4
4,13,5
...,...,...
18640,85,175
18641,85,9983
18642,85,194
18643,85,9982


In [48]:
bitacora[['Tarea', 'Viaje']] = bitacora[['Tarea', 'Viaje']].fillna(0).astype(int)

In [49]:
# Se selecionan las columnas de Bitacora que se van a usar
bitacorafil = bitacora[['Jerarquía',	'Tarea',	'Viaje', 'Parada', 'Vehículo', 'Novedad']]
# Se filtran los casos que no tiene numero de viaje ya que no tiene retoma
bitacorafil = bitacorafil[bitacorafil['Viaje'] != 0]

In [50]:
# Se compara las filas de los df, el resultado debe ser False
bitacora.shape[0]==bitacorafil.shape[0]

False

In [51]:
# Se agrega al merge anterior la informacion de bitacora
# Importante tener en cuenta los vehiculos que tengan mas de un caso en la bitacora, esto genera id duplicados.
mergebitacora = pd.merge(merge, bitacorafil, left_on=['Número de tarea vehículo_x', 'Número de viaje_x'], right_on=['Tarea', 'Viaje'], how='outer' )

In [52]:
# Revisar este resultado, ya que los valore sque no coinceden deven se iguales en el filtro
mergebitacora.shape[0]- merge.shape[0]

34

In [53]:
mergebitacora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18679 entries, 0 to 18678
Data columns (total 77 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Fecha_x                          18679 non-null  datetime64[ns]
 1   Día tipo_x                       18679 non-null  object        
 2   Designación de tarea vehículo_x  18679 non-null  object        
 3   Línea_x                          18679 non-null  object        
 4   Tipo de viaje corto_x            18679 non-null  object        
 5   Sentido_x                        13902 non-null  object        
 6   Tipo Kilómetros                  18679 non-null  object        
 7   Número de Vehículo               16165 non-null  object        
 8   Concesionario de Transporte_x    16117 non-null  object        
 9   Número de tarea vehículo_x       18679 non-null  object        
 10  Desde                            18679 non-null  object   

In [54]:
#mergebitacora[mergebitacora['id'].duplicated()]

Columna condicional 

In [55]:
# Se cambian los vehiculos del cierre por los de k2##
mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]
# Marca en la columnas Cambios los cambios que realiza el código
mergebitacora['condicion'] = [ np.nan if  pd.notna(x) else 'Se Cambia # V RF' for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]


In [56]:
# Función para actualizar los valores de Largo_y si Largo_x es mayor
def actualizar_largo(row):
    if pd.isna(row['Largo_y']):
        return row['Largo_x']
    elif row['Largo_x'] > row['Largo_y']:
        return row['Largo_x']
    else:
        return row['Largo_y']

In [57]:

def actualizar_largo_texto(row):
    if pd.isna(row['Largo_y']) and pd.isna(row['condicion']) :
        return 'Cambia largo por NAN'
    elif pd.isna(row['Largo_y']) and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia largo por NAN'
    elif row['Largo_x'] > row['Largo_y'] and pd.isna(row['condicion']):
        return 'Cambia x Mayor largo'
    elif row['Largo_x'] > row['Largo_y'] and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia x Mayor largo'
    else:
        if pd.notnull(row['condicion']):
            return row['condicion']
            


In [58]:
# Marcar las filas donde se realizan cambios en una nueva columna
mergebitacora['condicion'] = mergebitacora.apply(actualizar_largo_texto, axis=1)

In [59]:
# Aplicar la función a la columna Largo_y
mergebitacora['Largo_y'] = mergebitacora.apply(actualizar_largo, axis=1)

In [60]:
mergebitacora['condicion'].value_counts()

condicion
Se Cambia # V RF                         2332
Cambia x Mayor largo                      363
Se Cambia # V RF/Cambia x Mayor largo      20
Name: count, dtype: int64

In [67]:
# Cambia Blanco y Negro Masivo por BYN 
mergebitacora['Concesionario Programado'] = mergebitacora['Concesionario Programado'].replace('Blanco y Negro Masivo','BYN')

In [81]:
# Condiciones para la nueva columna
condicion1 = mergebitacora['Largo_x'] != mergebitacora['Largo_y']
condicion2 = mergebitacora['Número de Vehículo'] != mergebitacora['Número de secuencia de carro']
condicion3 = mergebitacora['Secuencia de arcos_x'] != mergebitacora['Secuencia de arcos_y']
condicion4 = mergebitacora['Concesionario de Transporte_x'] != mergebitacora['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['condicional2'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, parada y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionarios',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))

In [82]:
mergebitacora['condicional2'].value_counts()

condicional2
OK                                       9946
Revisar consesionario                    5540
Revisar vehiculo y consecionario         2603
Revisar largo y consecionario             243
Revisar largo                             236
Revisar numero de vehiculo                100
Revisar largo y vehiculo                    7
Revisar largo, parada y consecionario       4
Name: count, dtype: int64

In [ ]:
# Exporta a un archivo excel el Df
mergebitacora.to_excel('../output/prueba_mergebita.xlsx', index=False, sheet_name='k2_procesado')